# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [9]:
import requests
from bs4 import BeautifulSoup

### Send HTTP request to the target Website

In [10]:
url = 'https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/'
response = requests.get(url)

check the response status code

In [11]:
response.status_code

200

### Parse the response content with BeautifulSoap

In [12]:
soup = BeautifulSoup(response.text, 'html.parser')

# print(soup)

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

Repeat the extraction process for each page of products

In [13]:
list_of_products = []

page_numbers = soup.select('.wd-pagination li')
number_of_pages = int(page_numbers[-2].text)

for i in range(number_of_pages):
    # print(i)
    products = soup.select('.product')

    for product in products:
        tmp = dict()
        tmp['ProductTitle'] = product.select_one('.wd-entities-title').text
        prices = product.select('bdi')
        tmp['RegularPrice'] = int(prices[0].text.split('\xa0ден')[0].replace('.', ''))
        if len(prices) > 1:
            tmp['DiscountPrice'] = int(prices[1].text.split('\xa0ден')[0].replace('.', ''))
        tmp['ProductURL'] = product.select_one('.wd-entities-title a')['href']
        tmp['AddToCartURL'] = url + product.select_one('.add_to_cart_button')['href']
        list_of_products.append(tmp) 
        
    if i != number_of_pages-1:
        response = requests.get(soup.select_one('.next')['href'])
        soup = BeautifulSoup(response.text, 'html.parser')

# print(len(list_of_products))

### Create a pandas DataFrame with the scraped products

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame(list_of_products)
df

,ProductTitle,RegularPrice,ProductURL,AddToCartURL,DiscountPrice
0,Acer A315-23-A7KD,17590,https://clevershop.mk/product/acer-a315-23-a7kd/,https://clevershop.mk/product-category/mobilni...,NaN
1,Acer A315-23-R5P2,27490,https://clevershop.mk/product/acer-a315-23-r5p2/,https://clevershop.mk/product-category/mobilni...,NaN
2,ACER Aspire 1 A115-22,18999,https://clevershop.mk/product/acer-aspire-1-nx...,https://clevershop.mk/product-category/mobilni...,15999.0
3,Acer Aspire 3 A315-23-R26A,29990,https://clevershop.mk/product/acer-aspire-3-a3...,https://clevershop.mk/product-category/mobilni...,NaN
4,Acer Aspire 3 A315-58-33WK,24490,https://clevershop.mk/product/21498/,https://clevershop.mk/product-category/mobilni...,NaN
...,...,...,...,...,...
315,Monitor 27 Philips 272E1GAJ/00 VA 1ms 144Hz,12890,https://clevershop.mk/product/monitor-27-phili...,https://clevershop.mk/product-category/mobilni...,NaN
316,Philips 24″ 243V7QDSB,8390,https://clevershop.mk/product/philips-24%e2%80...,https://clevershop.mk/product-category/mobilni...,NaN
317,Philips 27″ 278E1A/00 4K UHD IPS,18990,https://clevershop.mk/product/hp-27%e2%80%b3-2...,https://clevershop.mk/product-category/mobilni...,NaN
318,Philips 279C9-00 MON LED 27″ 3840 x 2160 5Ms 6...,26990,https://clevershop.mk/product/philips-279c9-00...,https://clevershop.mk/product-category/mobilni...,NaN


Save the dataframe as `.csv`

In [16]:
df.to_csv("data/products_lab1.csv", index=False)